In [1]:
# Bibliotecas utilizadas
import requests
import json
import time
import pandas as pd

In [2]:
# getToken retorna um token de acesso para a api spotify
def getToken():
    print('Solicitando token...')
    url = "https://accounts.spotify.com/api/token"
    
    basic = "Basic ODY4NDUxY2Y5YWI2NDUwYjkwZDIwMmQzOTZhN2FiY2I6NzBlMjJjNGRmZGJkNDJlZWIyZDc1NmMzMDBiOThjMmI="

    payload = "grant_type=client_credentials&undefined="
    headers = {
        'Content-Type': "application/x-www-form-urlencoded",
        'Authorization': basic,
        'cache-control': "no-cache",
        'Postman-Token': "26d5613d-6954-4c55-b39a-b9387316f347"
        }

    response = requests.request("POST", url, data=payload, headers=headers)

    responseJson = json.loads(response.text)
    print('Token -> ' + responseJson['access_token'])
    return responseJson['access_token']


In [3]:
# Seta a autorização
def setHeaders():
    return {'Authorization': 'Bearer '+ getToken()}


In [4]:
# Função para aguardar o tempo para nova requisição
def waitTime(tempo):
    print('Aguardando liberação... Tempo estimado: ' + tempo/60 + 'min')
    time.sleep(tempo)
    print('Espera finalizada.')

In [5]:
# getCollectionMongo conecta ao mongo localhost
# cria o db tcc-spotify
# define a collection informada como parâmetro
def getCollectionMongo(collection):
    from pymongo import MongoClient
    client = MongoClient('localhost', 27017)
    banco = client['tcclista']
    return banco[collection]

## Realizando buscas das playlists

In [6]:
# define os países utilizados para realizar a busca
paises = ['africa','brazil', 'france']

In [ ]:
url = "https://api.spotify.com/v1/search?q=brazil&type=playlist&limit=10"
response = requests.request("GET", url)
print(response.headers['Date'])

print(response.status_code)

# response = json.loads(response.text)

# if 'error' in response:
#     print(response['error']['status'])

In [7]:
query_buscas = [
    'top50',
    'top+50',
    'the+best',
    'as+melhores',
    'le+eilleur',
    'die+este',
    'melhores+da+semana',
    'este+van+die+week',
    'meilleur+de+la+semaine'
]

In [13]:
# Acessa a api, realizando as buscas para cada pais e setando no banco
print('Inicio das buscas...')
header = setHeaders()
for busca in query_buscas:
    for pais in paises:
        query = "query="+pais+"&"+busca+"&type=playlist&limit=50"
        url = "https://api.spotify.com/v1/search?"+query
        response = ''
        while True:
            response = requests.request("GET", url, headers=header)

            if (response.status_code == 200):
                break

            if (response.status_code == 401):
                header = setHeaders()

            if (response.status_code == 429):
                waitTime(response.headers['Retry-After'])

        response = json.loads(response.text)
        playlists = response['playlists']['items']
        for track in playlists:
            track['country'] = pais
        collection = getCollectionMongo('playlists')
        collection.insert_many(playlists)

    
print('Busca finalizada.')
    

Inicio das buscas...
Solicitando token...
Token -> BQBA2dmtGDV9mgS09zk0sB9ZmC_h9CeQTNHXDtqsKmHer5I3EsYHe7F8MBIKLfVskOcqhqdgWAM4k4UNNvo
Busca finalizada.


## Verificando coleção

In [14]:
collection = getCollectionMongo('playlists')

In [15]:
df = pd.DataFrame(list(collection.find()))

In [16]:
df.head(11)

,_id,collaborative,country,external_urls,href,id,images,name,owner,primary_color,public,snapshot_id,tracks,type,uri
0,5ce73bc2e77ddf19678bae90,False,brazil,{'spotify': 'https://open.spotify.com/playlist...,https://api.spotify.com/v1/playlists/3aOeLwAN4...,3aOeLwAN4McOaFbmxdUkD8,"[{'height': None, 'url': 'https://pl.scdn.co/i...",Br azilian Rave Trance • Echoes Of The Universe,"{'display_name': 'psndoadan', 'external_urls':...",None,None,ODg5LDExYWVhOTJlNzBhODg5YzU2ZDQyZDUyNGVjZjJiMj...,{'href': 'https://api.spotify.com/v1/playlists...,playlist,spotify:playlist:3aOeLwAN4McOaFbmxdUkD8
1,5ce73bc5e77ddf19678baf5e,False,africa,{'spotify': 'https://open.spotify.com/playlist...,https://api.spotify.com/v1/playlists/3zgUlsiGv...,3zgUlsiGvkWiUropgDmDH2,"[{'height': 640, 'url': 'https://mosaic.scdn.c...",Africa – Toto,"{'display_name': 'Anne Braam', 'external_urls'...",None,None,NzM1LGM2NDMzZGQ2Mjk0Zjg1MmIzODI2Y2RiMTA4YTgwYj...,{'href': 'https://api.spotify.com/v1/playlists...,playlist,spotify:playlist:3zgUlsiGvkWiUropgDmDH2
2,5ce73bc5e77ddf19678baf5f,False,africa,{'spotify': 'https://open.spotify.com/playlist...,https://api.spotify.com/v1/playlists/37i9dQZF1...,37i9dQZF1DWX0o6sD1a6P5,"[{'height': None, 'url': 'https://pl.scdn.co/i...",Afro Hits,"{'display_name': 'Spotify', 'external_urls': {...",None,None,MTU1ODY1MTU2MCwwMDAwMDA1ZjAwMDAwMTZhZTZkZjYwOD...,{'href': 'https://api.spotify.com/v1/playlists...,playlist,spotify:playlist:37i9dQZF1DWX0o6sD1a6P5
3,5ce73bc5e77ddf19678baf60,False,africa,{'spotify': 'https://open.spotify.com/playlist...,https://api.spotify.com/v1/playlists/37i9dQZF1...,37i9dQZF1DWWW9iyuOPGds,"[{'height': None, 'url': 'https://pl.scdn.co/i...",Top Hits South Africa,"{'display_name': 'Spotify', 'external_urls': {...",None,None,MTU1ODQyNjQwNSwwMDAwMDFiMzAwMDAwMTZhZDk3M2NhOW...,{'href': 'https://api.spotify.com/v1/playlists...,playlist,spotify:playlist:37i9dQZF1DWWW9iyuOPGds
4,5ce73bc5e77ddf19678baf61,False,africa,{'spotify': 'https://open.spotify.com/playlist...,https://api.spotify.com/v1/playlists/5529YIpbJ...,5529YIpbJSNjU9BXFPqZI8,"[{'height': 640, 'url': 'https://mosaic.scdn.c...",Toto — Africa,"{'display_name': 'Jordy van den Brink', 'exter...",None,None,MjIxLDYyNDY3MDM2NjEzZDUxMjVkMzUyMzM4ZWIxZTdjNT...,{'href': 'https://api.spotify.com/v1/playlists...,playlist,spotify:playlist:5529YIpbJSNjU9BXFPqZI8
5,5ce73bc5e77ddf19678baf62,False,africa,{'spotify': 'https://open.spotify.com/playlist...,https://api.spotify.com/v1/playlists/37i9dQZF1...,37i9dQZF1DX9wXOYagay8L,"[{'height': None, 'url': 'https://pl.scdn.co/i...",Made In South Africa,"{'display_name': 'Spotify', 'external_urls': {...",None,None,MTU1ODY0ODc0MCwwMDAwMDEzODAwMDAwMTZhZTZiNDU4Zj...,{'href': 'https://api.spotify.com/v1/playlists...,playlist,spotify:playlist:37i9dQZF1DX9wXOYagay8L
6,5ce73bc5e77ddf19678baf63,False,africa,{'spotify': 'https://open.spotify.com/playlist...,https://api.spotify.com/v1/playlists/37i9dQZF1...,37i9dQZF1DX9TaUKtmYQ2X,"[{'height': None, 'url': 'https://pl.scdn.co/i...",Top Hits North Africa,"{'display_name': 'Spotify', 'external_urls': {...",None,None,MTU1ODQzODc5NiwwMDAwMDAxNTAwMDAwMTZhZGEzMGRkZj...,{'href': 'https://api.spotify.com/v1/playlists...,playlist,spotify:playlist:37i9dQZF1DX9TaUKtmYQ2X
7,5ce73bc5e77ddf19678baf64,False,africa,{'spotify': 'https://open.spotify.com/playlist...,https://api.spotify.com/v1/playlists/37i9dQZF1...,37i9dQZF1DWYkaDif7Ztbp,"[{'height': None, 'url': 'https://pl.scdn.co/i...",African Heat,"{'display_name': 'Spotify', 'external_urls': {...",None,None,MTU1ODA2OTY4NSwwMDAwMDBkMzAwMDAwMTZhYzQzMGFjZj...,{'href': 'https://api.spotify.com/v1/playlists...,playlist,spotify:playlist:37i9dQZF1DWYkaDif7Ztbp
8,5ce73bc5e77ddf19678baf65,False,africa,{'spotify': 'https://open.spotify.com/playlist...,https://api.spotify.com/v1/playlists/37i9dQZF1...,37i9dQZF1DXd0uyASpbU8w,"[{'height': None, 'url': 'https://pl.scdn.co/i...",New Music Friday South Africa,"{'display_name': 'Spotify', 'external_urls': {...",None,None,MTU1O

In [17]:
collection.estimated_document_count()

3751

In [18]:
df.country.value_counts()

brazil    1251
france    1250
africa    1250
Name: country, dtype: int64

## Removendo playlists duplicadas

In [19]:
for index, id_playlist in enumerate(df['id'], start=1):
    count = collection.count_documents({'id': id_playlist})
    if count > 1:
        cursor = collection.find({"id": id_playlist})
        collection.delete_one({"_id": list(cursor)[0]["_id"]})
        
        

In [20]:
collection.estimated_document_count()

155

## Realizando busca das musicas das playlists e setando em uma nova collection top musics

In [21]:
# Acessa a api, realizando as buscas para todos os tracks das playlists e setando no banco
header = setHeaders()
for playlist in collection.find():
    
    url = playlist["tracks"]["href"]
    pais = playlist["country"]
    response = ''
    while True:
        response = requests.request("GET", url, headers=header)

        if (response.status_code == 200):
            break

        if (response.status_code == 401):
            header = setHeaders()

        if (response.status_code == 429):
            waitTime(response.headers['Retry-After'])
            
    response = json.loads(response.text)
    musics = response['items']

    collectionBusca = getCollectionMongo('musicplaylist')
    for track in musics:
        try:
            track['track']['country'] = pais
            collectionBusca.insert_one(track['track'])
        except:
            print('Não possui faixa associada. País: %s' % pais)
            
        

print('Buscas finalizadas.')

Solicitando token...
Token -> BQAs4fIKlYIjmT_ysENgBCiY9H7gqxZaHXHGedSjdzENPXX7eLo4-jZ0EhClqVTsPZ-SnKn5DksfOGglGzo
Buscas finalizadas.


## Removendo músicas duplicadas

In [22]:
musicCollection = getCollectionMongo('musicplaylist')

In [23]:
musicCollection.estimated_document_count()

18889

In [26]:
df_musics = pd.DataFrame(list(musicCollection.find()))

In [27]:
for index, id_music in enumerate(df_musics['id'], start=1):
    count = musicCollection.count_documents({'id': id_music})
    if count > 1:
        cursor = musicCollection.find({"id": id_music})
        musicCollection.delete_one({"_id": list(cursor)[0]["_id"]})
        

In [28]:
musicCollection.estimated_document_count()

8723

## Limpando dados

In [29]:
# Acessando o mongo novamente, para obter a lista filtrada
musicCollection = getCollectionMongo('musicplaylist')
df_musics = pd.DataFrame(list(musicCollection.find()))
df_musics.head()

,_id,album,artists,available_markets,country,disc_number,duration_ms,episode,explicit,external_ids,...,id,is_local,linked_from,name,popularity,preview_url,track,track_number,type,uri
0,5ce74653e77ddf19678bf95a,"{'album_type': 'single', 'artists': [{'externa...",[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CA, C...",africa,1,193179,False,False,{'isrc': 'GBKPL1948237'},...,4ZpWpW8O3GhcCFye63rtwP,False,NaN,Oh Oh Oh (Lucie) - Popcaan Remix,46,https://p.scdn.co/mp3-preview/23e68f35fef50b75...,True,1,track,spotify:track:4ZpWpW8O3GhcCFye63rtwP
1,5ce74653e77ddf19678bf95c,"{'album_type': 'single', 'artists': [{'externa...",[{'external_urls': {'spotify': 'https://open.s...,[],africa,1,204956,False,False,{'isrc': 'QZES61997399'},...,3DaKjiludsRen7Y1t1Jm6G,False,NaN,SugarMummy,16,None,True,1,track,spotify:track:3DaKjiludsRen7Y1t1Jm6G
2,5ce74653e77ddf19678bf96a,"{'album_type': 'single', 'artists': [{'externa...",[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CA, C...",africa,1,179591,False,False,{'isrc': 'ZA56E1900563'},...,5l2OANZli43u5kAG1PxWtP,False,NaN,Get Up,37,https://p.scdn.co/mp3-preview/198a13d3d785686e...,True,2,track,spotify:track:5l2OANZli43u5kAG1PxWtP
3,5ce74653e77ddf19678bf977,"{'album_type': 'single', 'artists': [{'externa...",[{'external_urls': {'spotify': 'https://open.s...,"[AE, AR, AU, BO, BR, CA, CL, CO, CR, DO, EC, G...",africa,1,327852,False,False,{'isrc': 'USZ4V1900125'},...,1r2VCyUIuVKnNaViOXDGAN,False,NaN,Give Dem (feat. Kah-Lo),56,https://p.scdn.co/mp3-preview/fbdd2fb0aadaccec...,True,1,track,spotify:track:1r2VCyUIuVKnNaViOXDGAN
4,5ce74653e77ddf19678bf98a,"{'album_type': 'single', 'artists': [{'externa...",[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CA, C...",africa,1,227913,False,False,{'isrc': 'GBKPL1938929'},...,3thrLLxesR6sOSZJbmo7qd,False,NaN,Démo,48,https://p.scdn.co/mp3-preview/32b33e11775819c2...,True,1,track,spotify:track:3thrLLxesR6sOSZJbmo7qd


In [30]:
df_musics.country.value_counts()

brazil    3007
africa    2895
france    2821
Name: country, dtype: int64

In [31]:
len(df_musics)

8723

In [32]:
df_musics['artists'][0][0]

{'external_urls': {'spotify': 'https://open.spotify.com/artist/6mMtnxEQkYoY5FfJIQ9Rhb'},
 'href': 'https://api.spotify.com/v1/artists/6mMtnxEQkYoY5FfJIQ9Rhb',
 'id': '6mMtnxEQkYoY5FfJIQ9Rhb',
 'name': 'Runtown',
 'type': 'artist',
 'uri': 'spotify:artist:6mMtnxEQkYoY5FfJIQ9Rhb'}

In [33]:
df_album = pd.DataFrame(list(df_musics.album))

In [34]:
df_album.head()

,album_type,artists,available_markets,external_urls,href,id,images,name,release_date,release_date_precision,total_tracks,type,uri
0,single,[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CA, C...",{'spotify': 'https://open.spotify.com/album/66...,https://api.spotify.com/v1/albums/6647i8uAU1yI...,6647i8uAU1yIy6n3z8HzDn,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Oh Oh Oh (Lucie) (Popcaan Remix),2019-05-10,day,1.0,album,spotify:album:6647i8uAU1yIy6n3z8HzDn
1,single,[{'external_urls': {'spotify': 'https://open.s...,[],{'spotify': 'https://open.spotify.com/album/08...,https://api.spotify.com/v1/albums/08iEUAY3w6G8...,08iEUAY3w6G8ZA0WhWXmde,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",SugarMummy,2019-04-19,day,1.0,album,spotify:album:08iEUAY3w6G8ZA0WhWXmde
2,single,[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CA, C...",{'spotify': 'https://open.spotify.com/album/58...,https://api.spotify.com/v1/albums/58Yq2nQc0Np8...,58Yq2nQc0Np8veW1dsAO2F,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Jollification,2019-04-19,day,6.0,album,spotify:album:58Yq2nQc0Np8veW1dsAO2F
3,single,[{'external_urls': {'spotify': 'https://open.s...,"[AE, AR, AU, BO, BR, CA, CL, CO, CR, DO, EC, G...",{'spotify': 'https://open.spotify.com/album/2j...,https://api.spotify.com/v1/albums/2jicKj9QgiWd...,2jicKj9QgiWdoUpmAX0g4c,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Give Dem (feat. Kah-Lo),2019-05-01,day,1.0,album,spotify:album:2jicKj9QgiWdoUpmAX0g4c
4,single,[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CA, C...",{'spotify': 'https://open.spotify.com/album/2J...,https://api.spotify.com/v1/albums/2JKWKNAJuRJb...,2JKWKNAJuRJbXQpnyJj0TK,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Démo,2019-03-08,day,1.0,album,spotify:album:2JKWKNAJuRJbXQpnyJj0TK


In [35]:
df_musics.album = df_album['name']
df_musics['album_href'] = df_album['href']

In [36]:
df_musics.head()

,_id,album,artists,available_markets,country,disc_number,duration_ms,episode,explicit,external_ids,...,is_local,linked_from,name,popularity,preview_url,track,track_number,type,uri,album_href
0,5ce74653e77ddf19678bf95a,Oh Oh Oh (Lucie) (Popcaan Remix),[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CA, C...",africa,1,193179,False,False,{'isrc': 'GBKPL1948237'},...,False,NaN,Oh Oh Oh (Lucie) - Popcaan Remix,46,https://p.scdn.co/mp3-preview/23e68f35fef50b75...,True,1,track,spotify:track:4ZpWpW8O3GhcCFye63rtwP,https://api.spotify.com/v1/albums/6647i8uAU1yI...
1,5ce74653e77ddf19678bf95c,SugarMummy,[{'external_urls': {'spotify': 'https://open.s...,[],africa,1,204956,False,False,{'isrc': 'QZES61997399'},...,False,NaN,SugarMummy,16,None,True,1,track,spotify:track:3DaKjiludsRen7Y1t1Jm6G,https://api.spotify.com/v1/albums/08iEUAY3w6G8...
2,5ce74653e77ddf19678bf96a,Jollification,[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CA, C...",africa,1,179591,False,False,{'isrc': 'ZA56E1900563'},...,False,NaN,Get Up,37,https://p.scdn.co/mp3-preview/198a13d3d785686e...,True,2,track,spotify:track:5l2OANZli43u5kAG1PxWtP,https://api.spotify.com/v1/albums/58Yq2nQc0Np8...
3,5ce74653e77ddf19678bf977,Give Dem (feat. Kah-Lo),[{'external_urls': {'spotify': 'https://open.s...,"[AE, AR, AU, BO, BR, CA, CL, CO, CR, DO, EC, G...",africa,1,327852,False,False,{'isrc': 'USZ4V1900125'},...,False,NaN,Give Dem (feat. Kah-Lo),56,https://p.scdn.co/mp3-preview/fbdd2fb0aadaccec...,True,1,track,spotify:track:1r2VCyUIuVKnNaViOXDGAN,https://api.spotify.com/v1/albums/2jicKj9QgiWd...
4,5ce74653e77ddf19678bf98a,Démo,[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CA, C...",africa,1,227913,False,False,{'isrc': 'GBKPL1938929'},...,False,NaN,Démo,48,https://p.scdn.co/mp3-preview/32b33e11775819c2...,True,1,track,spotify:track:3thrLLxesR6sOSZJbmo7qd,https://api.spotify.com/v1/albums/2JKWKNAJuRJb...


In [40]:
results = spotify.search(q='artist:bruno mars', type='artist')
print(results)

SpotifyException: http status: 401, code:-1 - https://api.spotify.com/v1/search?q=artist%3Abruno+mars&limit=10&offset=0&type=artist:
 No token provided

##### Transformar a coluna artista em estilo da música e nome do artista

In [172]:
df_musics['artists_href'] = ''
df_musics['artists_name'] = ''
index = 0
for artista in df_musics.artists:

    try:
        df_musics['artists_name'][index] = artista[0]['name']
        df_musics['artists_href'][index] = artista[0]['href']
    except:
        df_musics['artists_name'][index] = artista['name']
        df_musics['artists_href'][index] = artista['href']
    
    index = index + 1

df_musics.head()

/home/adriano/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
/home/adriano/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
/home/adriano/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


,_id,album,artists,available_markets,country,disc_number,duration_ms,episode,explicit,external_ids,...,name,popularity,preview_url,track,track_number,type,uri,album_href,artists_href,artists_name
0,5ce7464ee77ddf19678bf743,Best of Belik,{'external_urls': {'spotify': 'https://open.sp...,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CA, C...",brazil,1,402306,False,False,{'isrc': 'UKACT1650525'},...,Avada Kadabra,54,https://p.scdn.co/mp3-preview/a1e82eb097066277...,True,1,track,spotify:track:2lBhQDBX4kbNrWQOS5HCk0,https://api.spotify.com/v1/albums/3ZffFmSUUl07...,https://api.spotify.com/v1/artists/7seaqlSfH73...,Belik Boom
1,5ce7464ee77ddf19678bf744,Toxic Generation's Music,[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CA, C...",brazil,1,394285,False,False,{'isrc': 'GBLV61604767'},...,Sequence - Original Mix,49,https://p.scdn.co/mp3-preview/34bfe7ff93cdae69...,True,3,track,spotify:track:22T6UPu3Ynx2Nk3e1GNtyg,https://api.spotify.com/v1/albums/7h5SFro3BZMd...,https://api.spotify.com/v1/artists/7t6kb9BaKLB...,Major7
2,5ce7464ee77ddf19678bf745,Tandava (Blazy & Gottinari Remix),[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CA, C...",brazil,1,312160,False,False,{'isrc': 'GBLV61641870'},...,Tandava - Blazy & Gottinari Remix,47,https://p.scdn.co/mp3-preview/47882f903972e1e0...,True,1,track,spotify:track:2BmwXAUDtEzKw4RGtHJu5B,https://api.spotify.com/v1/albums/7q4kYwG50Wmy...,https://api.spotify.com/v1/artists/4udXDlf1Vbs...,Shanti People
3,5ce7464ee77ddf19678bf746,"Shiva Style (Berg, Phanatic Remix)",[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CA, C...",brazil,1,422608,False,False,{'isrc': 'DEGA21702480'},...,"Shiva Style - Berg, Phanatic Remix",39,https://p.scdn.co/mp3-preview/348017c40c8446f9...,True,1,track,spotify:track:6Jf6SuWBMPpQydPRPAYy3n,https://api.spotify.com/v1/albums/66tKlCxHApV6...,https://api.spotify.com/v1/artists/053tej9oWvu...,Mandragora
4,5ce7464ee77ddf19678bf747,Sem Chão,[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AR, AT, AU, BE, BG, BH, BO, BR, CA, C...",brazil,1,247022,False,False,{'isrc': 'GBLV61642320'},...,Sem Chão,58,https://p.scdn.co/mp3-preview/f39d1d0c179bc875...,True,1,track,spotify:track:5irKKFoCUyKFq8rvtSBIYM,https://api.spotify.com/v1/albums/3mCC0lCqstMO...,https://api.spotify.com/v1/artists/2AasvmwafZP...,Mandragora


In [ ]:
df_ajuste = pd.DataFrame(list(musicCollection.find()))
df_musics.artists[0] = df_ajuste.artists[0][0]
df_musics.head()

In [ ]:
df_artists = pd.DataFrame(list(df_musics.artists))
df_artists.head()

In [ ]:
df_musics.artists = df_artists['name']
df_musics['artists_href'] = df_artists['href']
df_musics.head()

In [43]:
# Removendo colunas que não serão utilizadas
df_musics = df_musics.drop(columns=['available_markets', 'disc_number',
       'episode', 'explicit', 'external_ids','is_local', 'external_urls', 'preview_url', 'track',
       'track_number', 'type'])
df_musics.head()

,_id,album,artists,country,duration_ms,href,id,linked_from,name,popularity,uri,album_href
0,5ce74653e77ddf19678bf95a,Oh Oh Oh (Lucie) (Popcaan Remix),[{'external_urls': {'spotify': 'https://open.s...,africa,193179,https://api.spotify.com/v1/tracks/4ZpWpW8O3Ghc...,4ZpWpW8O3GhcCFye63rtwP,NaN,Oh Oh Oh (Lucie) - Popcaan Remix,46,spotify:track:4ZpWpW8O3GhcCFye63rtwP,https://api.spotify.com/v1/albums/6647i8uAU1yI...
1,5ce74653e77ddf19678bf95c,SugarMummy,[{'external_urls': {'spotify': 'https://open.s...,africa,204956,https://api.spotify.com/v1/tracks/3DaKjiludsRe...,3DaKjiludsRen7Y1t1Jm6G,NaN,SugarMummy,16,spotify:track:3DaKjiludsRen7Y1t1Jm6G,https://api.spotify.com/v1/albums/08iEUAY3w6G8...
2,5ce74653e77ddf19678bf96a,Jollification,[{'external_urls': {'spotify': 'https://open.s...,africa,179591,https://api.spotify.com/v1/tracks/5l2OANZli43u...,5l2OANZli43u5kAG1PxWtP,NaN,Get Up,37,spotify:track:5l2OANZli43u5kAG1PxWtP,https://api.spotify.com/v1/albums/58Yq2nQc0Np8...
3,5ce74653e77ddf19678bf977,Give Dem (feat. Kah-Lo),[{'external_urls': {'spotify': 'https://open.s...,africa,327852,https://api.spotify.com/v1/tracks/1r2VCyUIuVKn...,1r2VCyUIuVKnNaViOXDGAN,NaN,Give Dem (feat. Kah-Lo),56,spotify:track:1r2VCyUIuVKnNaViOXDGAN,https://api.spotify.com/v1/albums/2jicKj9QgiWd...
4,5ce74653e77ddf19678bf98a,Démo,[{'external_urls': {'spotify': 'https://open.s...,africa,227913,https://api.spotify.com/v1/tracks/3thrLLxesR6s...,3thrLLxesR6sOSZJbmo7qd,NaN,Démo,48,spotify:track:3thrLLxesR6sOSZJbmo7qd,https://api.spotify.com/v1/albums/2JKWKNAJuRJb...


In [44]:
df_musics.to_csv('backuplinkmusica.csv')

In [ ]:
print('Iniciando busca de features...')
print('Setando as chaves...')
keys = [
    'danceability',
    'energy',
    'loudness', 
    'speechiness',
    'acousticness',
    'instrumentalness',
    'liveness',
    'valence',
    'tempo'
]
print('Chaves setadas.')

# Cria um dicionário para inserir as chaves que desejo
print('Criando dicionário...')
infor = {}
infor = infor.fromkeys(keys)

print('Inserindo as chaves no dicionário...')
for key in infor.keys():
    infor[key] = []

print('Finalizado inserção das chaves no dicionário.')

print('Buscando e setando audio-features...')

basic1 = True

for index in range(0, len(df_musics)):
    
    print('Inserção %s' % index)
    id = df_musics['id'][index]
    
    while True:
        try:
            url = "https://api.spotify.com/v1/audio-features/" + id
            response = requests.request("GET", url, data="", headers=setHeaders())
            break
        except:
            print('err')
            
    track_info = json.loads(response.text)
    for key in infor.keys():
        infor[key].append(track_info[key])

print("Busca de audio-features finalizada.")


In [ ]:
dt

In [ ]:
for index in range(5654, len(df_musics)):
    
    print('Busca %s' % index)
    id = df_musics['id'][index]
    
    while True:
        try:
            url = "https://api.spotify.com/v1/audio-features/" + id
            response = requests.request("GET", url, data="", headers=setHeaders(basic1))
            break
        except:
            print('wait...')
            print(response.text)
            break
            
    track_info = json.loads(response.text)
    for key in infor.keys():
        infor[key].append(track_info[key])

print("Busca de audio-features finalizada.")

In [ ]:
len(infor['danceability'])

In [ ]:
len(df_musics)

In [ ]:
df_aux = pd.DataFrame(infor)
df_aux

In [ ]:
df_aux.to_csv('keys.csv')

In [ ]:
df_music_csv = pd.read_csv('dataset2_tcc.csv')
df_music_csv.head()

In [ ]:
for index, key in enumerate(infor.keys(), start=0):
    df_musics[key] = pd.Series(infor[key])
    
df_musics.head()


In [ ]:
len(df_musics)

In [ ]:
df_musics.to_csv('dataset2_tcc.csv')

In [ ]:
import pandas as pd
teste = pd.read_csv('dataset2_tcc.csv')
teste.head()

In [ ]:
teste.tail()

## Buscando o Genero do Artista

In [ ]:
df_musics['genres'] = ''
for index, href in enumerate(df_musics['artists_href'], start=0):
    print(index)
    print(href)
    
    while True:
        try:
            response = requests.request("GET", href, headers=setHeaders())
            response = json.loads(response.text)
            df_musics['genres'][index] = response['genres']
            break
        except:
            print('err')
    
df_musics.head()

In [ ]:
df_musics.head()

In [ ]:
for index, href in enumerate(df_musics['artists_href'], start=5654):
    print(index)
    print(href)
    print(df_musics['artists_href'][index])
    
    while True:
        try:
            response = requests.request("GET", df_musics['artists_href'][index], headers=setHeaders())
            response = json.loads(response.text)
            df_musics['genres'][index] = response['genres']
            break
        except:
            print('err')
    
df_musics.head()